# January 23, 2024 Session Notes

#### 1. Few Shots Example (Method 1)

In [28]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain_openai import ChatOpenAI
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, ChatPromptTemplate
import os 

In [29]:
set_llm_cache(InMemoryCache())

OPENAI_API_KEY_PERSONAL = os.getenv("OPENAI_API_KEY_PERSONAL")
model = ChatOpenAI(openai_api_key = OPENAI_API_KEY_PERSONAL)

In [30]:
template = "You are a helpful AI assisstant that translates complex legal language text into plain and simple terms"
sysmsg = SystemMessagePromptTemplate.from_template(template)

In [31]:
legal_text = """The provisions herein shall be severable, and if any provision or portion thereof is deemed invalid, illegal, 
or unenforceable by a court of competent jurisdiction, 
the remaining provisions or portions thereof shall remain in full force and effect to the maximum extent permitted by law."""

In [32]:
example_input_1 = HumanMessagePromptTemplate.from_template(legal_text)

In [33]:
plain_english_text = """The rules in this agreement can be separated. If a court decides that one rule or part of it is not valid, illegal, or cannot be enforced, 
the other rules will still apply and be enforced as much as they can under the law."""

In [34]:
example_output_1 = AIMessagePromptTemplate.from_template(plain_english_text)

In [35]:
my_legal_text = """The grantor, being the fee simple owner of the real property herein described, conveys and 
warrants to the grantee, his heirs and assigns, all of the grantor's right, title, and interest in and to the said property, 
subject to all existing encumbrances, liens, and easements, as recorded in the official records of the county, 
and any applicable covenants, conditions, and restrictions affecting the property, 
in consideration of the sum of [purchase price] paid by the grantee."""

In [36]:
my_prompt_templ = HumanMessagePromptTemplate.from_template(my_legal_text)

In [37]:
chat_prompt = ChatPromptTemplate.from_messages([sysmsg, example_input_1, example_output_1, my_prompt_templ])
prompt = chat_prompt.format_prompt().to_messages()

In [38]:
result = model.invoke (prompt)

In [39]:
print (result.content)

The person who currently owns the property is giving and guaranteeing to the person receiving it, along with their heirs and future owners, all of their rights and ownership in the property mentioned. However, there may be existing issues like loans, legal claims, or rights of way recorded in the county's official records that the new owner will have to deal with. There may also be certain rules and restrictions affecting the property. This transfer is happening because the new owner has paid a certain amount of money to the current owner.


#### 2. Few Shots Example (Method 2)

In [41]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain_openai import ChatOpenAI
import os 

In [42]:
few_shots = [
    {'Word':'Happy', 'Antonym':'Bad'},
    {'Word':'Hot', 'Antonym':'Cold'},
    {'Word':'Fast', 'Antonym':'Slow'},
    {'Word':'Brave', 'Antonym':'Coward'}
]

In [43]:
example_prompt = PromptTemplate(input_variables=["Word,Antonym"], template="Word: {Word} Antonym: {Antonym}")

In [44]:
dynamic_prompt = FewShotPromptTemplate(
    input_variables=['word'],
    examples=few_shots,
    example_prompt=example_prompt,
    prefix="I want antonym of the given word",
    suffix = "Query: {word}, Result:"
)

In [45]:
prompt = dynamic_prompt.format(word="Big")

In [46]:
result = model.invoke (prompt)

In [47]:
print (result.content)

Small


In [48]:
new_examples = {"Word": "Big", "Antonym":"Small"}

In [49]:
new_examples

{'Word': 'Big', 'Antonym': 'Small'}

In [50]:
few_shots

[{'Word': 'Happy', 'Antonym': 'Bad'},
 {'Word': 'Hot', 'Antonym': 'Cold'},
 {'Word': 'Fast', 'Antonym': 'Slow'},
 {'Word': 'Brave', 'Antonym': 'Coward'}]

In [51]:
few_shots.append(new_examples)

In [52]:
few_shots

[{'Word': 'Happy', 'Antonym': 'Bad'},
 {'Word': 'Hot', 'Antonym': 'Cold'},
 {'Word': 'Fast', 'Antonym': 'Slow'},
 {'Word': 'Brave', 'Antonym': 'Coward'},
 {'Word': 'Big', 'Antonym': 'Small'}]

#### 3. Output Parsers

In [13]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain.output_parsers import DatetimeOutputParser, OutputFixingParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, ChatPromptTemplate
import os 


In [14]:
set_llm_cache(InMemoryCache())

OPENAI_API_KEY_PERSONAL = os.getenv("OPENAI_API_KEY_PERSONAL")
model = ChatOpenAI(openai_api_key = OPENAI_API_KEY_PERSONAL)

In [15]:
output_parser = DatetimeOutputParser()

In [16]:
output_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1083-10-11T08:23:44.042931Z, 1511-09-15T14:37:13.172798Z, 1170-12-18T10:34:40.987190Z\n\nReturn ONLY this string, no other words!"

In [17]:
hum_msg = HumanMessagePromptTemplate.from_template(template="When did Neil Armstrong landed on moon? \n{my_desired_format}")

In [18]:
chat_prompt = ChatPromptTemplate.from_messages([hum_msg])

In [19]:
prompt = chat_prompt.format_prompt(my_desired_format=output_parser.get_format_instructions())

In [20]:
print (prompt)

messages=[HumanMessage(content="When did Neil Armstrong landed on moon? \nWrite a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0457-07-17T00:47:20.141598Z, 0816-12-07T21:27:19.231823Z, 2023-03-16T09:50:43.694527Z\n\nReturn ONLY this string, no other words!")]


In [21]:
result = model.invoke(prompt)

In [22]:
print(result.content)

1969-07-20T20:17:40.000000Z


In [23]:
type(result.content)

str

In [24]:
output_parser.parse(result.content)

datetime.datetime(1969, 7, 20, 20, 17, 40)